## this notebook is a test of nucdiff using some fake sequence data

In [2]:
import os, sys, io, random, subprocess
import string
import numpy as np
import pandas as pd
pd.set_option('display.width',600)
from Bio.Seq import Seq, MutableSeq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

sys.path.append('/home/tortoise/pybioviz/')
from pybioviz import utils

### make two test sequence and make gaps and some mutations in the second

In [52]:
rec = SeqIO.parse('ref_genomes/mbovis.fna',format='fasta').__next__()

s1=rec.seq[:50000] 
seq1 = SeqRecord(s1,id='test1',description='my ref')
#create altered seq

s2 = (s1[:200]+ 10*'ATATATATATAGCGCGCGATGCTAG' +s1[200:400]+s1[400:500]+s1[600:18000]+s1[20000:30000]+s1[35000:40000]).tomutable()
s2[200] = 'T'
s2[1000] = 'A'

bases = ['A', 'T', 'G', 'C']
for i in range(10):
    position = random.randrange(len(s2))
    base = bases[random.randrange(len(bases))]
    while s2[position] == base:
        position = random.randrange(len(s2))
    print(position)
    s2[position] = base


seq2 = SeqRecord(s2,id='test2',description='my query')

SeqIO.write(seq1,'test1.fna','fasta')
SeqIO.write(seq2,'test2.fna','fasta')

13387
8734
26014
5678
18888
5132
30347
12320
29034
32696


1

In [32]:
nuc = utils.align_nucmer('test1.fna','test2.fna')

nucmer --maxgap=500 --mincluster=100 --coords -p nucmer test1.fna test2.fna


In [33]:
nuc

,S1,E1,S2,E2,LEN 1,LEN 2,IDENT,TAG1,TAG2
0,1,500,1,500,500,500,99.80,test1,test2
1,601,18000,501,17900,17400,17400,99.99,test1,test2
2,20001,30001,17901,27901,10001,10001,100.00,test1,test2
3,35000,40000,27900,32900,5001,5001,100.00,test1,test2


### run nucdiff and find the differences!

* run nucdiff on the 2 samples with test1 as ref and find deletions plus snps

In [53]:
featurekeys = ['ID', 'Name', 'del_len', 'ins_len', 'query_bases', 'ref_bases', 'query_coord', 'start', 'end']

def features_to_dataframe(features, cds=False):
    """Get features from a biopython seq record object into a dataframe
    Args:
        features: bio seqfeatures
       returns: a dataframe with a row for each cds/entry.
      """

    #preprocess features
    allfeat = []
    for (item, f) in enumerate(features):
        x = f.__dict__
        q = f.qualifiers
        #print(q)
        x.update(q)
        d = {}
        d['start'] = f.location.start
        d['end'] = f.location.end
        #d['strand'] = f.location.strand
        for i in featurekeys:
            if i in x:
                if type(x[i]) is list:
                    d[i] = x[i][0]
                else:
                    d[i] = x[i]
        allfeat.append(d)

    df = pd.DataFrame(allfeat,columns=featurekeys)

    return df

feats = utils.gff_to_features('Nucdiff_test12/results/first_ref_struct.gff')
feat_structs = features_to_dataframe(feats)
deletions = feat_structs[(feat_structs.Name=='deletion') | (feat_structs.Name=='insertion')]


feats_snps = utils.gff_to_features('Nucdiff_test12/results/first_ref_snps.gff')
feat_snps = features_to_dataframe(feats_snps)
snps = feat_snps[feat_snps.Name=='substitution']


snpdels = pd.concat([deletions, snps])
snpdels

,ID,Name,del_len,ins_len,query_bases,ref_bases,query_coord,start,end
0,SV_1,insertion,NaN,248,NaN,NaN,201-448,199,200
2,SV_3,deletion,100,NaN,NaN,NaN,750,500,600
3,SV_4,deletion,2000,NaN,NaN,NaN,18150,18000,20000
4,SV_5,deletion,4998,NaN,NaN,NaN,28151,30001,34999
0,SNP_1,substitution,NaN,NaN,A,G,5133-5133,4982,4983
1,SNP_2,substitution,NaN,NaN,C,A,5679-5679,5528,5529
2,SNP_3,substitution,NaN,NaN,G,C,8735-8735,8584,8585
3,SNP_4,substitution,NaN,NaN,T,C,12321-12321,12170,12171
4,SNP_5,substitution,NaN,NaN,G,A,13388-13388,13237,13238
5,SNP_6,substitution,NaN,NaN,A,G,18889-18889,20738,20739
